### 정수 인코딩(Integer Encoding)

- 단어 토큰화 또는 형태소 토큰화를 수행했다면 각 단어에 고유한 정수부여
- 중복이 혀용되지 않는 모든 단어 집합 생성
- 이를 단어 집합이라 하며 이를 기반으로 문서를 정수로 인코딩

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import torch
import urllib.request
from tqdm import tqdm
from collections import Counter
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv("IMDB Dataset.csv")

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df['sentiment'].value_counts().plot(kind='bar')

#### 이진 물류같은 경우 레이블을 1,0으로 변경한다

In [ ]:
df['sentiment']=df['sentiment'].replace(['negative','positive'],[0, 1])

### 훈련데이터와 테스트 데이터로 분류한다
- test_size를 통하여 비율 산정
- stratify에 레이블에 해당하는 데이터를 부여하면 훈련 데이터,테스트 데이터를 나눌 때 레이블의 비율을 균일하게 나누어 준다

In [ ]:
X_data=df['review']
y_data=df['sentiment']
X_train,X_test,y_train,y_test=train_test_split(X_data,y_data,test_size=0.5,random_state=0,stratify=y_data)

X_train[0]

#### 토큰화 처리

In [ ]:
sample=word_tokenize(X_train[0])
print(sample)

In [ ]:
lower_sample=[word.lower() for word in sample] #소문지화 전처리

#### 데이터 토큰화

In [ ]:
def tokenize(sentences):
    tokenized_sentences=[]
    for sentence in tqdm(sentences):
        tokenized_sent=word_tokenize(sentence)
        tokenized_sent=[word.lower() for word in tokenized_sent]
        tokenized_sentences.append(tokenized_sent)
    return tokenized_sentences

In [ ]:
tokenized_X_train=tokenize(X_train)
tokenized_X_test=tokenize(X_test)


In [ ]:
for sent in tokenized_X_train[:2]:
    print(sent)

#### 정수 인코딩을 위하여 **vocabulary** 를 만들어야한다
- 훈련데이터의 모든 단어들을 모은다
- 이를 위해 파이썬에 있는 카운터 묘듈사용
- 입력으로 들어온 단어들의 빈도수를 체크

In [ ]:
word_list=[]
for sent in tokenized_X_train:
    for word in sent:
        word_list.append(word)
word_counts=Counter(word_list)
print('총 단어수 ',len(word_counts))

In [ ]:
print(word_counts)

In [ ]:
print('훈련데이터에서 단어 the의 등장횟수:',word_counts['the'])

- 등장빈도수에 맞추어 정렬

In [ ]:
vocab=sorted(word_counts,key=word_counts.get,reverse=True)
print('등장 빈도수 상위 10개 단어')
print(vocab[0:10])

- vocab 사이즈 축소를 위해 통계적인 정보 활용

In [ ]:
threshold = 3
total_cnt = len(word_counts) # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합
for key,value in word_counts.items():
    total_freq=total_freq+value
    if(value< threshold):
        rare_cnt=rare_cnt+1
        rare_freq=rare_freq+value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [ ]:
# 전체 단어 개수 중 빈도 수  1이하인 단어는 제거
vocab_size=total_cnt-rare_cnt
vocab=vocab[:vocab_size]
print('단어 집합의 크기:',len(vocab))

In [ ]:
print(vocab)

In [ ]:
word_to_index={}
word_to_index['<PAD>']=0
word_to_index['<UNK>']=1

In [ ]:
for index,word in enumerate(vocab):
    word_to_index[word]=index+2

In [ ]:
print(word_to_index)

In [ ]:
vocab_size = len(word_to_index)
print('패딩 토큰과 UNK 토큰을 고려한 단어 집합의 크기 :', vocab_size)

In [ ]:
print('단어 <PAD>와 맵핑되는 정수 :', word_to_index['<PAD>'])
print('단어 <UNK>와 맵핑되는 정수 :', word_to_index['<UNK>'])
print('단어 the와 맵핑되는 정수 :', word_to_index['the'])

In [ ]:
word_to_index['bridesmaid']

- UNK 이 있는 이유는 등장빈도수가 2개이하인 것 삭제 했기 때문에 정수로 변환하였기 때문에 key에러가 발생할 수 있다.
- 토큰화된 단어들 중 word_to_index에서 단어와 정수가 매핑된 것에서 키에러가 발생할 수 있는데 이러한 경우 UNK토큰으로 매핑한다.
-> 'UNK' => 'Unknown' => 모르는 단어가 등장했을 경우 맵핑하는 용도로 사용되는 스페셜 토큰.

In [ ]:
def texts_to_sequences(tokenized_X_data, word_to_index):
  encoded_X_data = []
  for sent in tokenized_X_data:
    index_sequences = []
    for word in sent:
      try:
          index_sequences.append(word_to_index[word])
      except KeyError:
          index_sequences.append(word_to_index['<UNK>'])
    encoded_X_data.append(index_sequences)
  return encoded_X_data

In [ ]:
print(tokenized_X_train[0])

In [ ]:
print(tokenized_X_train[1])

In [ ]:
len(tokenized_X_train[0])

In [ ]:
len(tokenized_X_train[1])

In [ ]:
encoded_X_train = texts_to_sequences(tokenized_X_train, word_to_index)
encoded_X_test = texts_to_sequences(tokenized_X_test, word_to_index)

In [ ]:
print(encoded_X_train[0])

In [ ]:
# 상위 샘플 2개 출력
for sent in encoded_X_train[:2]:
  print(sent)

In [ ]:
index_to_word = {}
for key, value in word_to_index.items():
    index_to_word[value] = key

In [ ]:
print(word_to_index)
print(index_to_word)

In [ ]:
decoded_sample = [index_to_word[word] for word in encoded_X_train[0]]
print('기존의 첫번째 샘플 :', tokenized_X_train[0])
print('복원된 첫번째 샘플 :', decoded_sample)

In [ ]:
print('리뷰의 최대 길이 :',max(len(review) for review in encoded_X_train))
print('리뷰의 평균 길이 :',sum(map(len, encoded_X_train))/len(encoded_X_train))
plt.hist([len(review) for review in encoded_X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

### 딥러닐 모델이 데이터를 처리하는 묶음 단위에서는 길이가 동일해야 딥러닝 모델이 행렬로 인식할 수 있어 병렬처리를 할 수 있다.
- 길이를 똑같게 해주는 작업을 패딩이라고 한다.

In [ ]:
# 숫자안에 숫자보다 짧은 샘플이 몇개인지 계산해주는 함수
def below_threshold_len(max_len, nested_list):
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

In [ ]:
max_len = 500
below_threshold_len(max_len, encoded_X_train)

In [ ]:
# 최대길이를 입력하면 최대 길이 보다 짧은 것들은 최대길이가 되게 0을 채우고 넘으면 최대길이 만큼되게 뒤를 자른다 
def pad_sequences(sentences, max_len):
  features = np.zeros((len(sentences), max_len), dtype=int)
  for index, sentence in enumerate(sentences):
    if len(sentence) != 0:
      features[index, :len(sentence)] = np.array(sentence)[:max_len]
  return features

In [ ]:
padded_X_train = pad_sequences(encoded_X_train, max_len=max_len)
padded_X_test = pad_sequences(encoded_X_test, max_len=max_len)

In [ ]:
print('훈련 데이터의 크기 :', padded_X_train.shape)
print('테스트 데이터의 크기 :', padded_X_test.shape)

In [ ]:
padded_X_train[:5]

In [ ]:
len(padded_X_train[0])

In [ ]:
print(padded_X_train[0])